Cryptocurrency trading bot.

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        cryptocurrency/crypto_logger_base.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger base class.

# Library imports.
from cryptocurrency.resampling import resample
from binance.client import Client
from abc import abstractmethod, ABC
from time import sleep, time
from os.path import exists, join
from os import mkdir

import pandas as pd

class Crypto_logger_base(ABC):
    def __init__(self, interval='15s', delay=4.7, buffer_size=3000, directory='crypto_logs', 
                 log_name='crypto_log', raw=False):
        """
        :param interval: OHLCV interval to log. Default is 15 seconds.
        :param delay: delay between Binance API requests. Minimum calculated was 4.7 seconds.
        :param buffer_size: buffer size to avoid crashing on memory accesses.
        :param directory: the directory where to output the logs.
        :param log_name: name of the log file.
        :param raw: whether the log dumps raw (instantaneous) or OHLCV data.
        """
        self.interval = interval
        self.delay = delay
        self.buffer_size = buffer_size
        self.directory = directory
        self.raw = raw

        self.log_name = join(self.directory, log_name + '.txt')
        self.log_screened_name = join(self.directory, log_name + '_screened.txt')

        if not exists(self.directory):
            mkdir(self.directory)

    #self.get_from_file(log_name=self.log_name, from_raw=False)
    #self.get_from_file(log_name=self.input_log_name, from_raw=self.load_from_ohlcv)
    def get_from_file(self, log_name, from_raw=False):
        if from_raw:
            dataset = pd.read_csv(log_name, header=0, index_col=0)
        else:
            dataset = pd.read_csv(log_name, header=[0, 1], index_col=0)
        dataset.index = pd.DatetimeIndex(dataset.index)
        return dataset.sort_index(axis='index')

    @abstractmethod
    def get(self, **kwargs):
        raise NotImplementedError()

    @abstractmethod
    def screen(self, **kwargs):
        raise NotImplementedError()

    def put(self, dataset):
        dataset = dataset.copy().reset_index()
        if self.raw:
            dataset = dataset.drop_duplicates(subset=['symbol', 'count'], 
                                              keep='first', ignore_index=True)
        else:
            dataset = dataset.drop_duplicates(keep='last', ignore_index=True)

        if 'date' in dataset.columns:
            min_index_int = dataset[dataset['date'] == self.min_index].index[0]
            dataset = dataset.set_index('date')
        if not self.raw:
            dataset = resample(dataset, self.interval)
        if 'date' in dataset.columns:
            dataset = dataset.iloc[min_index_int:]

        dataset = dataset.tail(self.buffer_size)
        dataset.to_csv(self.log_name)
        self.min_index = dataset.index[0]
        return dataset

    def start(self, append=False, roll=0):
        """Main logger loop."""
        print('Starting crypto logger.')

        if exists(self.log_name) and 'output' in self.log_name:
            self.dataset = self.get_from_file(log_name=self.log_name, from_raw=False)
            self.dataset = self.dataset.tail(self.buffer_size)
        else:
            self.dataset = self.get()

        self.min_index = self.dataset.index[-1]
        self.dataset = self.put(self.dataset)

        while True:
            try:
                dataset = pd.concat([self.dataset, self.get()], axis='index', join='outer')
            except (KeyboardInterrupt, SystemExit):
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            try:
                self.dataset = self.put(dataset)
            except (KeyboardInterrupt, SystemExit):
                print('Saving latest complete dataset...')
                self.dataset = self.put(dataset)
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            try:
                if exists(self.log_screened_name):
                    dataset_screened_old = \
                        pd.read_csv(self.log_screened_name, index_col=0, header=0)
                else:
                    dataset_screened_old = None
                dataset_screened = self.screen(self.dataset)
                if dataset_screened is not None:
                    if roll != 0:
                        if append and exists(self.log_screened_name):
                            dataset_screened = \
                                pd.concat([dataset_screened_old, dataset_screened], axis='index')
                            dataset_screened = \
                                dataset_screened.drop_duplicates(subset=['symbol'], keep='last')
                        dataset_screened = dataset_screened.tail(roll)
                        dataset_screened.to_csv(self.log_screened_name)
                    elif append:
                        dataset_screened.to_csv(self.log_screened_name, mode='a')
                    else:
                        dataset_screened.to_csv(self.log_screened_name)
            except (KeyboardInterrupt, SystemExit):
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            sleep(self.delay)
        print('Crypto logger process done.')

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        cryptocurrency/crypto_logger_output.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger output for arbitrary intervals.

# Library imports.
#from cryptocurrency.crypto_logger_base import Crypto_logger_base
from cryptocurrency.indicators import filter_in_market, screen_one
from os.path import exists, join

import pandas as pd
pd.options.mode.chained_assignment = None

class Crypto_logger_output(Crypto_logger_base):
    def __init__(self, delay=10, interval_input='15s', interval='15s', buffer_size=100, 
                 input_log_name='input'):
        """
        :param delay: delay between Binance API requests. Minimum calculated was 4.7 seconds.
        :param interval_input: OHLCV interval from input log. Default is 15 seconds.
        :param interval: OHLCV interval to log. Default is 15 seconds.
        :param buffer_size: buffer size to avoid crashing on memory accesses.
        :param input_log_name: the directory where to take the logs from.
        """
        self.data_before = pd.DataFrame()
        input_log_name = 'crypto_' + input_log_name + '_log_'
        self.load_from_ohlcv = interval_input != interval
        super().__init__(interval=interval, delay=delay, buffer_size=buffer_size, 
                         directory='crypto_logs', log_name='crypto_output_log_' + interval, 
                         raw=False)

        self.input_log_name = \
            join(self.directory, input_log_name + interval_input + '.txt')
        self.input_log_screened_name = \
            join(self.directory, input_log_name + interval_input + '_screened.txt')

    def screen(self, dataset):
        if exists(self.input_log_screened_name):
            input_filtered = pd.read_csv(self.input_log_screened_name, header=0, index_col=0)
            input_filter = set(input_filtered['symbol'].tolist())
            old_columns = set(dataset.columns.get_level_values(0).tolist())
            new_columns = list(input_filter & old_columns)
            dataset = dataset[new_columns]
            #dataset.columns = dataset.columns.swaplevel(0, 1)
            #dataset = dataset.rename(columns={'base_volume': 'volume'})
            #dataset.columns = dataset.columns.swaplevel(0, 1)
            assets = filter_in_market(screen_one, dataset)
            #dataset.columns = dataset.columns.swaplevel(0, 1)
            #dataset = dataset.rename(columns={'volume': 'base_volume'})
            #dataset.columns = dataset.columns.swaplevel(0, 1)
            return input_filtered[input_filtered['symbol'].isin(assets)]
        else:
            return None

    def resample_from_raw(self, df):
        df = df[['symbol', 'close', 'rolling_base_volume', 'rolling_quote_volume']]
        df['base_volume'] = df['rolling_base_volume'].copy()
        df['quote_volume'] = df['rolling_quote_volume'].copy()
        df = df.pivot_table(index=['date'], columns=['symbol'], 
                            values=['close', 'rolling_base_volume', 
                                    'rolling_quote_volume', 
                                    'base_volume', 'quote_volume'], 
                            aggfunc={'close': ['first', 'max', 'min', 'last'], 
                                     'base_volume': 'max', 'quote_volume': 'max', 
                                     'rolling_base_volume': 'max', 
                                     'rolling_quote_volume': 'max'})
        df.columns = pd.MultiIndex.from_tuples([('_'.join(col[:2]), col[2]) for col in df.columns.values], 
                                               names=('pair', 'symbol'))
        df = df.rename(columns={'close_first': 'open', 'close_max': 'high', 
                                'close_min': 'low', 'close_last': 'close', 
                                'base_volume_max': 'base_volume', 
                                'quote_volume_max': 'quote_volume', 
                                'rolling_base_volume_max': 'rolling_base_volume', 
                                'rolling_quote_volume_max': 'rolling_quote_volume'}, 
                       level=0)
        df['base_volume'] = df['base_volume'].fillna(method='pad')
        df['base_volume'].iloc[0] = 0
        df['quote_volume'] = df['quote_volume'].fillna(method='pad')
        df['quote_volume'].iloc[0] = 0
        df['rolling_base_volume'] = df['rolling_base_volume'].fillna(method='pad')
        df['rolling_base_volume'].iloc[0] = 0
        df['rolling_quote_volume'] = df['rolling_quote_volume'].fillna(method='pad')
        df['rolling_quote_volume'].iloc[0] = 0
        df = df.sort_index().iloc[1:]
        df.columns = df.columns.swaplevel(0, 1)
        return df

    def get(self):
        dataset = self.get_from_file(log_name=self.input_log_name, 
                                     from_raw=not self.load_from_ohlcv)
        if not self.load_from_ohlcv:
            dataset = self.resample_from_raw(dataset)
        return dataset.tail(2)

Main.

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        crypto_logger_output_1min.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger output for the 1 minute interval.

# Library imports.
#from cryptocurrency.crypto_logger_output import Crypto_logger_output

crypto_logger_output_1min = Crypto_logger_output(delay=44, 
                                                 interval_input='30s', 
                                                 interval='1min', 
                                                 buffer_size=1500, 
                                                 input_log_name='output')
crypto_logger_output_1min.start(append=False, roll=1000)

In [ ]:
import pandas as pd
input_log_name = 'crypto_logs/crypto_input_log_15s.txt'
df = pd.read_csv(input_log_name, header=0, index_col=0)
df.index = pd.DatetimeIndex(df.index)
df = df.sort_index(axis='index')
#df = crypto_logger_output_1min.resample_from_raw(df)
df

In [ ]:
import pandas as pd
#crypto_output_log_1min = crypto_logger_output_1min.log_name
crypto_output_log_1min = 'crypto_logs/crypto_output_log_1min.txt'
df_1min = pd.read_csv(crypto_output_log_1min, header=[0, 1], index_col=0)
df_1min.index = pd.DatetimeIndex(df_1min.index)
df_1min = df_1min.sort_index(axis='index')
df_1min['BTCUSDT'].tail(20)

In [ ]:
dataset = crypto_logger_output_1min.dataset
dataset

In [ ]:
dataset['BTCUSDT']

In [ ]:
dataset['ZILBUSD']

In [ ]:
def filter_in_market(function, dataset):
    def f(x):
        x = x.loc[:,~x.columns.duplicated()]
        return function(x)
    tickers_list = dataset.columns.get_level_values(0).unique().tolist()
    return pd.Series([ticker for ticker in tickers_list if f(dataset[ticker])], dtype='str')

def get_positive_trend_strength_trigger(data):
    ADX = data.ta.adx(talib=True)
    return (ADX['ADX_14'] < 0.20).iloc[-3] & (ADX['ADX_14'] > 0.20).iloc[-2]

def get_not_negative_trend_strength_trigger(data):
    ADX = data.ta.adx(length=14, lensig=8, talib=True)
    return ((ADX['DMP_14'] > ADX['DMN_14']) & (ADX['ADX_14'] > 0.30)).iloc[-1]

def get_not_negative_rebound_trigger(data):
    CCI = data.ta.cci(length=22, talib=True)
    MFI = data.ta.mfi(length=11, talib=True)
    return ((CCI > 0) | (MFI > 20)).iloc[-1]

def get_positive_choppiness_trigger(data):
    CHOP = data.ta.chop(talib=True)
    return CHOP.iloc[-1] < 38.2

def get_positive_phase_trigger(data):
    MACD = data.ta.macd(talib=True)
    histogram = MACD['MACDs_12_26_9'] - MACD['MACD_12_26_9']
    return ((histogram > histogram.shift(1)) | \
            (MACD['MACD_12_26_9'] > MACD['MACDs_12_26_9'])).iloc[-1]

def get_positive_phase_trigger(data):
    MACD = data.ta.macd(talib=True)
    histogram = MACD['MACDs_12_26_9'] - MACD['MACD_12_26_9']
    return ((histogram.iloc[-2] > histogram.iloc[-2]) or \
            (MACD['MACD_12_26_9'].iloc[-1] > MACD['MACDs_12_26_9'].iloc[-1]))

In [ ]:
filter_in_market(get_positive_phase_trigger, df)

In [ ]:
get_positive_phase_trigger(df['BTCUSDT'])

In [ ]:
from tqdm import tqdm

def filter_in_market(function, dataset):
    def f(x):
        x = x.loc[:,~x.columns.duplicated()]
        return function(x)
    tickers_list = dataset.columns.get_level_values(0).unique().tolist()
    return pd.Series([ticker for ticker in tqdm(tickers_list, unit=' ticker') if f(dataset[ticker])], dtype='str')

def get_heikin_ashi_trigger(ticker):
    def get_trend_strength_entry_trigger(heikin_ashi_dataset):
        ADX = heikin_ashi_dataset.ta.adx(talib=True)
        return (ADX['ADX_14'] < 0.20).iloc[-3] and (ADX['ADX_14'] > 0.20).iloc[-2]

    def get_not_negative_rebound_trigger(heikin_ashi_dataset):
        CCI = heikin_ashi_dataset.ta.cci(talib=True)
        return CCI.iloc[-1] < 100

    def get_positive_choppiness_trigger(heikin_ashi_dataset):
        CHOP = heikin_ashi_dataset.ta.chop(talib=True)
        return CHOP.iloc[-1] < 38.2

    def get_buy_trigger(heikin_ashi_dataset):
        return get_not_negative_rebound_trigger(heikin_ashi_dataset) and \
               (get_positive_choppiness_trigger(heikin_ashi_dataset) or \
                get_trend_strength_entry_trigger(heikin_ashi_dataset))

    heikin_ashi = ticker.ta.ha(talib=True)
    heikin_ashi_dataset = heikin_ashi.rename(columns={'HA_open': 'open', 
                                                      'HA_high': 'high', 
                                                      'HA_low': 'low', 
                                                      'HA_close': 'close'})
    try:
        trigger = get_buy_trigger(heikin_ashi_dataset)
    except:
        trigger = False
    return trigger

tickers_list = filter_in_market(get_heikin_ashi_trigger, dataset)
tickers_list

In [ ]:
heikin_ashi = dataset['BTCUSDT'].ta.ha(talib=True)
heikin_ashi_dataset = heikin_ashi.rename(columns={'HA_open': 'open', 
                                                  'HA_high': 'high', 
                                                  'HA_low': 'low', 
                                                  'HA_close': 'close'})
ADX = heikin_ashi_dataset.ta.adx(talib=True)
ADX[['DMP_14', 'DMN_14']]

In [ ]:
import pandas as pd
crypto_output_log_15s = 'crypto_logs/crypto_output_log_15s.txt'
df_15s = pd.read_csv(crypto_output_log_15s, header=[0, 1], index_col=0)
df_15s.index = pd.DatetimeIndex(df_15s.index)
df_15s

In [ ]:
import pandas as pd

crypto_exchange_info = 'crypto_logs/crypto_exchange_info.txt'
exchange_info = pd.read_csv(crypto_exchange_info, header=0, index_col=0)
exchange_info

In [ ]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion_table import get_conversion_table
from cryptocurrency.conversion import convert_price, get_base_asset_from_pair

authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info)

conversion_table.sort_values(by='rolling_traded_USDT_volume', ascending=False).reset_index(drop=True).head(50)

In [1]:
symbol = 'NEBLBUSD'

In [ ]:
import pandas as pd
crypto_output_log_1d = 'crypto_logs/crypto_output_log_1d.txt'
df_1d = pd.read_csv(crypto_output_log_1d, header=[0, 1], index_col=0)
df_1d.index = pd.DatetimeIndex(df_1d.index)
df_1d

In [2]:
import pandas as pd
crypto_output_log_30s = 'crypto_logs/crypto_output_log_30s.txt'
df_30s = pd.read_csv(crypto_output_log_30s, header=[0, 1], index_col=0)
df_30s.index = pd.DatetimeIndex(df_30s.index)
df_30s

symbol,1INCHBTC,1INCHBUSD,1INCHUSDT,AAVEBNB,AAVEBTC,AAVEBUSD,AAVEETH,AAVEUSDT,ACABTC,ACABUSD,...,ZILBTC,ZILBUSD,ZILETH,ZILEUR,ZILTRY,ZILUSDT,ZRXBTC,ZRXBUSD,ZRXETH,ZRXUSDT
,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,...,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume
date,,,,,,,,,,,,,,,,,,,,,
2022-10-24 13:52:30,1554.7,8744.8,8102.8,1.624,27.092,50.178,4.911,62.565,2230.22,1021.47,...,5.363197,477611.757728,104.646853,5340.624922,2.027540e+06,4.351362e+06,2.047123,23777.6795,8.870887,767815.0400
2022-10-24 13:53:00,1554.7,8744.8,8102.8,1.624,27.092,50.178,4.911,62.565,2230.22,1021.47,...,5.363197,477611.757728,104.646853,5340.624922,2.027540e+06,4.351362e+06,2.047123,23777.6795,8.870887,767815.0400
2022-10-24 13:53:30,1554.7,8744.8,8102.8,1.624,27.092,50.178,4.911,62.565,2230.22,1021.47,...,5.363197,477611.757728,104.646853,5340.624922,2.027540e+06,4.351362e+06,2.047123,23777.6795,8.870887,767815.0400
2022-10-24 13:54:00,1554.7,8744.8,8102.8,1.624,27.092,50.178,4.911,62.565,2230.22,1021.47,...,5.363197,477611.757728,104.646853,5340.624922,2.027540e+06,4.351362e+06,2.047123,23777.6795,8.870887,767815.0400
2022-10-24 13:54:30,1554.7,8744.8,8102.8,1.624,27.092,50.178,4.911,62.565,2230.22,1021.47,...,5.363197,477611.757728,104.646853,5340.624922,2.027540e+06,4.351362e+06,2.047123,23777.6795,8.870887,767815.0400
2022-10-24 13:55:00,1554.7,8744.8,8102.8,1.624,27.092,50.178,4.911,62.565,2230.22,1021.47,...,5.363197,477611.757728,104.646853,5340.624922,2.027540e+06,4.351362e+06,2.047123,23777.6795,8.870887,767815.0400
2022-10-24 13:55:30,1554.7,8744.8,8102.8,1.624,27.092,50.178,4.911,62.565,2230.22,1021.47,...,5.363197,477611.757728,104.646853,5340.624922,2.027540e+06,4.351362e+06,2.047123,23777.6795,8.870887,767815.0400
2022-10-24 13:56:00,1554.7,8744.8,8102.8,1.624,27.092,50.178,4.911,62.565,2230.22,1021.47,...,5.363197,477611.757728,104.646853,5340.624922,2.027540e+06,4.351362e+06,2.047123,23777.6795,8.870887,767815.0400


In [3]:
import pandas as pd
crypto_output_log_15s = 'crypto_logs/crypto_output_log_15s.txt'
df_15s = pd.read_csv(crypto_output_log_15s, header=[0, 1], index_col=0)
df_15s.index = pd.DatetimeIndex(df_15s.index)
df_15s

symbol,1INCHBTC,1INCHBUSD,1INCHUSDT,AAVEBNB,AAVEBTC,AAVEBUSD,AAVEETH,AAVEUSDT,ACABTC,ACABUSD,...,ZILBTC,ZILBUSD,ZILETH,ZILEUR,ZILTRY,ZILUSDT,ZRXBTC,ZRXBUSD,ZRXETH,ZRXUSDT
,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,...,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume
date,,,,,,,,,,,,,,,,,,,,,
2022-10-24 13:57:15,161688.3,1228281.9,3874490.5,1930.678,33780.838,23330.486,2226.309,162383.048,2522413.02,8446464.05,...,5.410523,477611.757728,104.646853,5340.624922,2.084627e+06,4.351362e+06,2.047123,23777.6795,8.870887,767815.0400
2022-10-24 13:57:30,161688.3,1228281.9,3839597.3,1930.678,33780.838,23330.486,2226.309,162383.048,2522413.02,8446464.05,...,5.410523,477611.757728,104.646853,5340.624922,2.084627e+06,4.351362e+06,2.047123,23777.6795,8.870887,767815.0400
2022-10-24 13:57:45,161600.5,1228281.9,3839597.3,1930.678,33780.838,23301.352,2226.309,162369.358,2522413.02,8446464.05,...,5.410523,477611.757728,104.646853,5340.624922,2.084627e+06,4.351362e+06,2.047123,23777.6795,8.870887,767815.0400
2022-10-24 13:58:00,161896.3,1227520.7,3804191.0,1930.678,33779.692,23306.165,2226.309,162516.212,2522413.02,8446464.05,...,5.410523,477611.757728,104.646853,5340.624922,2.084627e+06,4.351362e+06,2.047123,23777.6795,8.870887,767815.0400
2022-10-24 13:58:15,161896.3,1227520.7,3767225.7,1930.678,33780.408,23306.165,2226.309,162519.623,2522413.02,8446464.05,...,5.410523,477611.757728,104.646853,5340.624922,2.084627e+06,4.351362e+06,2.047123,23777.6795,8.870887,767815.0400
2022-10-24 13:58:30,161896.3,1225815.3,3732443.7,1930.678,33777.516,23306.165,2226.309,162482.246,2522413.02,8446464.05,...,5.409845,477611.757728,104.646853,5340.624922,2.084627e+06,4.351362e+06,2.047123,23777.6795,8.870887,767815.0400
2022-10-24 13:58:45,161850.9,1225815.3,3732443.7,1930.678,33777.516,23303.721,2226.309,162482.246,2522413.02,8446464.05,...,5.409845,477611.757728,104.646853,5340.624922,2.084627e+06,4.351362e+06,2.047123,23777.6795,8.870887,767815.0400
2022-10-24 13:59:00,161834.5,1223512.7,3697355.2,1930.678,33777.516,23303.721,2226.309,162482.246,2522413.02,8446464.05,...,5.409845,477611.757728,104.646853,5340.624922,2.092804e+06,4.351362e+06,2.047123,23777.6795,8.870887,767815.0400


In [4]:
df_15s[symbol]

,base_volume,close,high,low,open,quote_volume,rolling_base_volume,rolling_quote_volume
date,,,,,,,,
2022-10-24 13:57:15,723424.8,2.017,2.017,2.017,2.017,1.518020e+06,723424.8,1.518020e+06
2022-10-24 13:57:30,723424.8,2.017,2.017,2.017,2.017,1.518020e+06,723424.8,1.518020e+06
2022-10-24 13:57:45,723424.8,2.024,2.024,2.017,2.017,1.518020e+06,723424.8,1.518020e+06
2022-10-24 13:58:00,724923.2,2.016,2.017,2.016,2.017,1.521018e+06,724923.2,1.521018e+06
2022-10-24 13:58:15,724923.2,2.017,2.017,2.017,2.017,1.521018e+06,724923.2,1.521018e+06
2022-10-24 13:58:30,724923.2,2.017,2.017,2.017,2.017,1.521018e+06,724923.2,1.521018e+06
2022-10-24 13:58:45,724923.2,2.017,2.017,2.017,2.017,1.521018e+06,724923.2,1.521018e+06
2022-10-24 13:59:00,724923.2,2.017,2.017,2.017,2.017,1.521018e+06,724923.2,1.521018e+06
2022-10-24 13:59:15,724923.2,2.016,2.017,2.016,2.017,1.521018e+06,724923.2,1.521018e+06


In [5]:
import pandas as pd
crypto_output_log_1min = 'crypto_logs/crypto_output_log_1min.txt'
df_1min = pd.read_csv(crypto_output_log_1min, header=[0, 1], index_col=0)
df_1min.index = pd.DatetimeIndex(df_1min.index)
df_1min

symbol,1INCHBTC,1INCHBUSD,1INCHUSDT,AAVEBNB,AAVEBTC,AAVEBUSD,AAVEETH,AAVEUSDT,ACABTC,ACABUSD,...,ZILBTC,ZILBUSD,ZILETH,ZILEUR,ZILTRY,ZILUSDT,ZRXBTC,ZRXBUSD,ZRXETH,ZRXUSDT
,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,...,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume
date,,,,,,,,,,,,,,,,,,,,,
2022-10-23 12:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.993593,238331.139300,50.725583,10851.363651,1.465550e+06,2.805033e+06,1.199237,14753.4615,6.901583,536219.7798
2022-10-23 12:51:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.993593,238331.139300,50.725583,10851.363651,1.465550e+06,2.805734e+06,1.199237,14753.4615,6.901583,536245.3096
2022-10-23 12:52:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.994276,238331.139300,50.725583,10851.363651,1.465550e+06,2.805798e+06,1.200394,14753.4615,6.901583,536272.7608
2022-10-23 12:53:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.994276,238369.721452,50.725583,10851.363651,1.465550e+06,2.805814e+06,1.200394,14753.4615,6.901583,536289.8505
2022-10-23 12:54:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.994531,238369.721452,50.725583,10851.363651,1.466676e+06,2.806050e+06,1.200482,14753.4615,6.901583,536303.0945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-24 13:45:00,-624.4,-1357.5,-20991.1,0.000,0.000,0.000,0.000,-10.255,0.00,0.00,...,5.341503,477824.083748,104.753771,5325.909730,2.022948e+06,4.348677e+06,1.963956,23825.8133,8.451823,759749.9680
2022-10-24 13:46:00,-250.5,-1581.5,-10350.6,-3.266,-6.320,-59.609,0.000,-394.277,0.00,0.00,...,5.344642,476794.383423,104.753771,5325.909730,2.017891e+06,4.333526e+06,1.963956,23778.5107,8.451823,758671.9314


In [6]:
df_1min[symbol].tail(20)

,base_volume,close,high,low,open,quote_volume,rolling_base_volume,rolling_quote_volume
date,,,,,,,,
2022-10-24 13:30:00,-670.2,2.012,2.012,2.010,2.010,-1498.0615,721013.1,1.513993e+06
2022-10-24 13:31:00,316.5,2.017,2.017,2.017,2.017,627.9193,721329.6,1.514621e+06
2022-10-24 13:32:00,89.7,2.012,2.012,2.012,2.012,179.1786,721419.3,1.514800e+06
2022-10-24 13:33:00,-25.3,2.012,2.012,2.012,2.012,-54.1420,721394.0,1.514746e+06
2022-10-24 13:34:00,45.9,2.017,2.017,2.015,2.015,92.5343,721439.9,1.514838e+06
2022-10-24 13:35:00,-500.0,2.017,2.017,2.017,2.017,-1070.0732,720939.9,1.513768e+06
2022-10-24 13:36:00,-1367.1,2.017,2.017,2.017,2.017,-2931.7039,719572.8,1.510837e+06
2022-10-24 13:37:00,-2.5,2.017,2.017,2.014,2.014,-33.5693,719570.3,1.510803e+06
2022-10-24 13:38:00,28.9,2.012,2.012,2.012,2.012,58.1468,719599.2,1.510861e+06


In [ ]:
import pandas as pd
crypto_output_log_5min = 'crypto_logs/crypto_output_log_5min.txt'
df_5min = pd.read_csv(crypto_output_log_5min, header=[0, 1], index_col=0)
df_5min.index = pd.DatetimeIndex(df_5min.index)
df_5min

In [ ]:
df_5min[symbol].tail(20)

In [ ]:
df = df_1min.copy()
df

In [ ]:
df.columns = df.columns.swaplevel(0, 1)
df = df.stack(level=1).reset_index(level=1)
df_temp = df[['base_volume', 'quote_volume', 
              'rolling_base_volume', 'rolling_quote_volume']]
df_temp[df_temp['pair'] == 'base_volume'].iloc[:,1:] = \
    df_temp[df_temp['pair'] == 'rolling_base_volume'].iloc[:,1:].diff(1) + \
    df_temp[df_temp['pair'] == 'base_volume'].iloc[:,1:].shift(1440)
df_temp[df_temp['pair'] == 'quote_volume'].iloc[:,1:] = \
    df_temp[df_temp['pair'] == 'rolling_quote_volume'].iloc[:,1:].diff(1) + \
    df_temp[df_temp['pair'] == 'quote_volume'].iloc[:,1:].shift(1440)
df = df.reset_index().pivot_table(index=['date'], columns=['pair'], 
                                  values=df.columns[1:])
df

In [ ]:
df = df_1min.copy()
df = df.iloc[-1440:]
df_old = df.copy().iloc[:-10]
df.columns = df.columns.swaplevel(0, 1)
df = df.sort_index(axis='columns').drop(columns=['base_volume', 'quote_volume'])
df.columns = df.columns.swaplevel(0, 1)
df

In [ ]:
df[symbol]

In [ ]:
df = df_1min.copy()
df = df.iloc[-1440:]
df_old = df.copy().iloc[:-10]
df.columns = df.columns.swaplevel(0, 1)
df = df.sort_index(axis='columns').drop(columns=['rolling_base_volume', 'rolling_quote_volume'])
df.columns = df.columns.swaplevel(0, 1)
df

In [ ]:
df.columns

In [ ]:
grouped = df.groupby('pair', axis='columns', level=1)
grouped

In [ ]:
df[symbol]

In [ ]:
grouped = df.groupby('pair', axis='columns').describe()
grouped

In [ ]:
grouped['mean']

In [ ]:
df.groupby(level=0, axis=1).apply(lambda x: x[x.name].apply(lambda x: x, axis=1))

In [ ]:
df.groupby(level='symbol', axis='index')['base_volume']

In [ ]:
df.groupby(level='pair', axis='columns').apply(lambda x: x)

In [ ]:
df.groupby(level=['symbol', 'pair'], axis='columns').agg(lambda x: x)

In [ ]:
df.groupby('pair', axis=1).apply(lambda x: x[x.name == 'base_volume'])

In [ ]:
df.groupby('pair', axis=1, level=1).apply(lambda x: x.name)

In [ ]:
df.loc[:,(slice(None), 'base_volume')]

In [ ]:
df = df_1min.copy()
idx = pd.IndexSlice
#df_base_volume = df.loc[:,idx[:,'base_volume']]
#df_quote_volume = df.loc[:,idx[:,'quote_volume']]
#df_rolling_base_volume = df.loc[:,idx[:,'rolling_base_volume']]
#df_rolling_quote_volume = df.loc[:,idx[:,'rolling_quote_volume']]
df_volume = df.loc[:,idx[:,['base_volume', 'quote_volume', 'rolling_quote_volume', 'rolling_quote_volume']]].copy()
#df_base_volume = df_rolling_base_volume.droplevel(1, axis=1).diff(1) + df_base_volume.droplevel(1, axis=1).shift(1440)
#df_quote_volume = df_rolling_quote_volume.droplevel(1, axis=1).diff(1) + df_quote_volume.droplevel(1, axis=1).shift(1440)
#df = pd.merge(left=df, right=df_rolling_base_volume, how='right')
#df = pd.merge(left=df, right=df_rolling_quote_volume, how='right')
#df.loc[:,(slice(None), 'base_volume')] = df_base_volume
#df.loc[:,(slice(None), 'quote_volume')] = df_quote_volume
#df.loc[:,idx[:,'base_volume']] = \
#    df.loc[:,idx[:,'rolling_base_volume']].diff(1) + \
#    df.loc[:,idx[:,'base_volume']].shift(1440)
#df.loc[:,idx[:,'quote_volume']] = \
#    df.loc[:,idx[:,'rolling_quote_volume']].diff(1) + \
#    df.loc[:,idx[:,'quote_volume']].shift(1440)
#df['BTCUSDT'].tail(10)
df_volume #.loc[:,idx[:,'base_volume']] + df_volume.loc[:,idx[:,'rolling_base_volume']]

In [ ]:
df = df_1min.copy()
t1 = time()
df.iloc[:,df.columns.get_level_values(1) == 'base_volume'] = \
    df.xs('rolling_base_volume', axis=1, level=1).diff(1) + \
    df.xs('base_volume', axis=1, level=1).shift(1440)
df.iloc[:,df.columns.get_level_values(1) == 'quote_volume'] = \
    df.xs('rolling_quote_volume', axis=1, level=1).diff(1) + \
    df.xs('quote_volume', axis=1, level=1).shift(1440)
t2 = time()
print('It took', t2 - t1, 'seconds')
df

In [ ]:
df[symbol]

In [ ]:
from time import time

def recalculate_volumes(df):
    df = df.stack(level=1)
    df.reset_index(level=1, inplace=True)
    df[df['pair'] == 'base_volume'].iloc[:,1:] = \
        df[df['pair'] == 'rolling_base_volume'].iloc[:,1:].diff(1) + \
        df[df['pair'] == 'base_volume'].iloc[:,1:].shift(1440)
    df[df['pair'] == 'quote_volume'].iloc[:,1:] = \
        df[df['pair'] == 'rolling_quote_volume'].iloc[:,1:].diff(1) + \
        df[df['pair'] == 'quote_volume'].iloc[:,1:].shift(1440)
    df.reset_index(inplace=True)
    df = df.pivot_table(index=['date'], columns=['pair'], values=df.columns[1:], sort=False)
    #df.columns = df.columns.swaplevel(0, 1)
    #df = df[['open', 'high', 'low', 'close', 
    #         'base_volume', 'quote_volume', 
    #         'rolling_base_volume', 
    #         'rolling_quote_volume']]
    #df.columns = df.columns.swaplevel(0, 1)
    return df

df = df_1min.copy()
t1 = time()
df = recalculate_volumes(df)
t2 = time()
print('It took', t2 - t1, 'seconds')
df

In [ ]:
df[symbol]

In [ ]:
df1[symbol]

In [ ]:
from binance.client import Client
client = Client()
df = pd.DataFrame(client.get_ticker())
df

In [ ]:
df[df['symbol'] == symbol]['volume']

In [ ]:
df[df['symbol'] == symbol]['quoteVolume']

In [ ]:
df = pd.DataFrame(client.get_ticker())
float(df[df['symbol'] == symbol]['volume'].iloc[-1]) * float(df[df['symbol'] == symbol]['lastPrice'].iloc[-1])

In [ ]:
float(df[df['symbol'] == symbol]['lastQty'].iloc[-1]) * float(df[df['symbol'] == symbol]['lastPrice'].iloc[-1])

In [ ]:
#symbol = 'NEBLBUSD'
from cryptocurrency.ohlcv import download_pair
df = download_pair(client, symbol, interval='1m', period=2000)
#df = df.loc[:df_1min.index[-1]]
df.tail(10)

In [ ]:
quote_volume = df['quote_volume'] * df['close']
quote_volume

In [ ]:
new_rolling_volume = quote_volume.rolling('1440min').sum()
new_rolling_volume.tail(10)

In [ ]:
new_standard_volume = (new_rolling_volume.diff(1) + quote_volume.shift(1440))
new_standard_volume.tail(10)

In [ ]:
import numpy as np
np.convolve((new_rolling_volume).rolling('1440min').agg(np.sum), np.array([1, 0]), 'same')

In [ ]:
from pandas import read_csv

crypto_output_log_15s = 'crypto_logs/crypto_output_log_15s.txt'
df = read_csv(crypto_output_log_15s, header=[0, 1], index_col=0)
df

In [ ]:
df['NEBLBUSD']['volume']

In [ ]:
import pandas as pd
pd.infer_freq(df.asfreq('1min').index)

In [ ]:
df.index.freq == pd.tseries.frequencies.to_offset('1min')

In [ ]:
df['BTCUSDT'].drop(columns=['volume']).iloc[-100:].plot(figsize=(20, 10))

In [ ]:
# Run this from local machine:
#apt update
#apt install -y openssh-server sshfs vde2
#mkdir -p /home/samuel/workspace/crypto_logs/
#dpipe /usr/lib/openssh/sftp-server = ssh sam@154.12.239.24 sshfs :/home/samuel/workspace/crypto_logs/ /home/sam/workspace/crypto_logs -o slave &

In [ ]:
import pandas as pd
input_filtered = pd.read_csv(crypto_logger_output_1min.input_log_screened_name, header=None, 
                             index_col=0, names=['date', 'symbol', 'lastPrice'])
input_filter = input_filtered['symbol']
#input_filter = set(input_filter.tolist())
input_filter.isin(df['baseAsset'])

In [ ]:
from pandas import read_csv

crypto_input_log_1min = 'crypto_logs/crypto_input_log_1min.txt'
df = pd.read_csv(crypto_input_log_1min, header=0, index_col=0)
df

In [ ]:
df.columns

In [ ]:
symbol = 'BTCUSDT'
interval = '1m'
period = 2880

from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.ohlcv import download_pair

authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client

df = download_pair(client=client, symbol=symbol, interval=interval, period=period)
df

In [ ]:
import pandas as pd
df.astype('float32').memory_usage().sum()

In [ ]:
df.memory_usage().sum()